In [1]:
# 이미지있으면 이미지처리 -> 관련키워드는 따로
# 자격요건 넣기
# 랜덤으로 timesleep 주기
# 막혔을때 예외처리
# 스킬 추가

In [80]:
# conda install beautifulsoup4
# conda install selenium
import time
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import random
import re
import numpy as np

In [3]:
def remove_tags(string):
    return re.sub('<.+?>', '', str(string), 0).strip()

Data = pd.DataFrame({}, columns=[\
    'title','job', 'cname','duties','skills', 'preference', 'welfare', 'loc',\
    'rec type','url'])

In [5]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')        ## Head-less 설정
#options.add_argument('--no-sandbox')
#options.add_argumetn('--blink-setting=imagesEnabled=false') # 이미지 로딩 안함
#options.add_argument('--nute-audio') # 음소거
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-site-isolation-trials")
#options.add_argument('--disable-setuid-sandbox')
options.add_argument('--incognito') # 시크릿모드
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64)\
    AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.0.0 Safari/537.36') # user와 같은 설정 주기
chromedriver = '../data/chromedriver'

In [7]:
#driver.quit()
driver = webdriver.Chrome(chromedriver, options=options) # 크롬 드라이버 설정

In [8]:
base_url = 'https://www.jobkorea.co.kr/'
driver.get(base_url)
search_job = driver.find_element('xpath', '//*[@id="stext"]')
word = '데이터 사이언티스트'
search_job.send_keys(word+Keys.ENTER)
time.sleep(5) # 랜덤으로
html = driver.page_source

In [9]:
page_soup = BeautifulSoup(html, 'html.parser')
recruit_items = page_soup.find_all('li', {'class':'list-post'}) # list-post / list-post active

In [96]:
####### 각 채용공고당 진행
# for-loop start
import sys
import copy

#for recruit in recruits:
recruits = recruit_items[5].find('a', {'class':'title dev_view'})
rec_url = recruits['href']
base_url = 'https://www.jobkorea.co.kr'
search_url = base_url + rec_url
print (search_url)
#r = requests.get(search_url)
driver.get(search_url)
stime = 1+ 3*random.random()
time.sleep(stime)
html_job = BeautifulSoup(driver.page_source, 'html.parser')

https://www.jobkorea.co.kr/Recruit/GI_Read/38778532?Oem_Code=C1&logpath=1&stext=데이터 사이언티스트&listno=6


In [97]:
iframe = html_job.find('iframe', {'title':'상세요강'}) # html 내의 document (iframe) 처리
driver.switch_to.frame('gib_frame') # iframe으로 옮겨가기
stime = 1.+ 3*random.random()
time.sleep(stime)
html_iframe = BeautifulSoup(driver.page_source, 'html.parser')

In [98]:
title = remove_tags(html_job.find('title'))
job = word
company = remove_tags(html_job.find('span', {'class':'coName'}))
html_str = str(html_iframe)
url = search_url
print ('title : ', title)
print ('job : ', job)
print ('company : ', company)
print ('url : ', url)

# keywords and location
keyword = ''
loc = ''
assoc_keyword = html_job.find('article', {'id':'artKeywordSearch'})
for key in assoc_keyword.find_all('li'):
    if str(key).find('btnLocal') > -1:
        loc = remove_tags(key).replace('&gt;', '>')
        continue
    keyword += remove_tags(key) + ', '
print ('keyword : ', keyword)
print ('loc : ', loc)

# rec type
rec = ''
apply_qual = remove_tags(html_job.find('dl', {'class':'tbList'})).replace('\n','')
apply_qual1 = apply_qual.find('경력')
apply_qual2 = apply_qual.find('학력')
if apply_qual2 >= 0:
    rec = apply_qual[apply_qual1+2:apply_qual2].replace(' ','')
else:
    rec = apply_qual[apply_qual1+2:].replace(' ','')
print ('rec type : ', rec)

title :  카비 채용 - [CARVI] 데이터사이언티스트 총괄 | 잡코리아
job :  데이터 사이언티스트
company :  ㈜카비
url :  https://www.jobkorea.co.kr/Recruit/GI_Read/38778532?Oem_Code=C1&logpath=1&stext=데이터 사이언티스트&listno=6
keyword :  우수벤처기업 공동채용관, IT분야 직무채용관, DBA·데이터베이스, 데이터마이닝, 웹프로그래머, MySQL, Python, SQL, 빅데이터·AI(인공지능), 인공지능(AI), 빅데이터, 머신러닝(ML), 딥러닝, 데이터분석, 데이터사이언스, 데이터사이언티스트, 데이터애널리스트, data, Data Scientist, data science, 솔루션·SI·CRM·ERP, 모바일앱개발, 소프트웨어개발, 솔루션, BigData, DataMining, 
loc :  서울 > 금천구
rec type :  경력(7년이상)


In [99]:
# 상세 채용 정보 내용
duties = ''
skills = ''
preference = ''
welfare = ''


content = html_iframe.find('div', {'class':'detailed-summary-contents'})
img = html_iframe.find('div', {'class':'detailReadImg'})
if str(content) != 'None':
    content_table = html_iframe.find('div', {'class':'recruitment'})
    content_lines = content_table.find('div', {'class':'recruitment-item'})

    ps = content.find_all('p')
    for ip in ps:
        # duties
        str_ip = remove_tags(ip)
        if '담당업무' in str(ip): # duties
            case = 1
        elif any([needs in str(ip) for needs in ['필수', '자격']]): # skills
            case = 2
        elif '우대' in str(ip): # preference
            case = 3
        elif any([i in str(ip) for i in ['복지', '복리']]): # welfare
            case = 4
        elif any([i in str(ip) for i in ['전형절차']]):
            case = -1
        try:
            if case == 1: # duties
                duties += str_ip + '\n'
            elif case == 2: # skills
                skills += str_ip + '\n'
            elif case == 3: # preference
                preference += str_ip + '\n'
            elif case == 4: # welfare
                welfare += str_ip + '\n'
            elif case == -1: # others
                print ('other : ',str_ip)
        except NameError:
            pass
elif str(img) != 'None':
    #table = img.find('div', {'class':'tbDetailWrap tbDetailType_3 ct va tbBgNone'})
    #print (table)
    pass
    
        
        
print ('duties : \n',duties)
print ('skills : \n',skills)
print ('preference : \n',preference)
print ('welfare : \n',welfare)

duties : 
 
skills : 
 
preference : 
 
welfare : 
 


In [45]:
welfare = ''
content = html_iframe.find('div', {'class':'detailed-summary-contents'})
print(content.find_all('p'))


print ('welfare : ',welfare)


#print (Data)

[<p class="heading"><i class="icon-header dev-case-type case2"></i>모집분야 및 자격요건</p>, <p><b>서비스분석</b><br class="default"/>( 0명 ) </p>, <p><b>담당업무</b></p>, <p>ㆍ추천/개인화 모형 개발</p>, <p>ㆍ고객 행동 예측 및 세그멘테이션</p>, <p>ㆍ전사 데이터 분석을 통한 이슈 파악 및 과제 해결</p>, <p>ㆍfeature store 및 데이터 마트를 위한 변수 발굴 및 정의</p>, <p><br/></p>, <p><b>필수 경험과 역량</b> </p>, <p>ㆍ데이터 분석 및 모델링을 통해 비즈니스 문제를 해결한 경험이 있는 분</p>, <p>ㆍ모형개발 분야 3년 이상의 업무 경험이 있는 분</p>, <p>ㆍ통계학, 컴퓨터공학, 산업공학, 수학 및 기타 유사 학문을 전공한 분</p>, <p><br/></p>, <p><b>우대사항</b></p>, <p>ㆍ추천시스템/NLP/Graph Neural Network에 대한 이해가 있는 분<br/></p>, <p>ㆍTensorFlow/PyTorch와 같은 딥러닝 프레임워크를 잘 활용하는 분</p>, <p>ㆍHadoop/Spark 등을 활용한 대용량 데이터 처리 및 분석 경험이 있는 분</p>, <p>ㆍ데이터 엔지니어 및 타 팀과의 커뮤니케이션이 원활한 분</p>, <p>ㆍ데이터 마트 설계 및 구축 경험(운영)이 있는 분</p>, <p><br/></p>, <p><b>공통 우대 사항</b></p>, <p>ㆍ국가유공자 및 장애인 등 취업보호대상자는 관계 법령에 따라 우대합니다. </p>, <p>   (입사지원 시 증빙서류를 첨부하는 경우에 해당하며, 증빙서류는 포트폴리오 란에 </p>, <p>    첨부해주시길 바랍니다.)</p>, <p><br/></p>, <p class="heading"><i class="icon-header dev-case-type case2"></i>근무 관련 정보</p>, <p>

In [101]:
# Data에 저장
length = Data.shape[0]
#Data.drop([length-1], inplace=True)

#length = Data.shape[0]
Data.loc[length] = [title, job, company, duties, skills, preference, welfare, loc, rec, url]
Data

,title,job,cname,duties,skills,preference,welfare,loc,rec type,url
0,카비 채용 - [CARVI] 데이터사이언티스트 총괄 | 잡코리아,데이터 사이언티스트,㈜카비,,,,,서울 > 금천구,경력(7년이상),https://www.jobkorea.co.kr/Recruit/GI_Read/387...


In [102]:
import openpyxl
Data.to_excel('../output/DataScience_0721_1.xlsx')

In [76]:
print (url)

https://www.jobkorea.co.kr/Recruit/GI_Read/38778532?Oem_Code=C1&logpath=1&stext=데이터 사이언티스트&listno=6


In [95]:
# Data초기화
Data = pd.DataFrame({}, columns=[\
    'title','job', 'cname','duties','skills', 'preference', 'welfare', 'loc',\
    'rec type','url'])

In [111]:
driver.quit()